In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='2', take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_multi_lstm import *

In [4]:
graph_name = 'SpectrogramMultiLSTM'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

#g_cnfg.conv1_n_filters = 16
#g_cnfg.conv1_kernel_size = 5
#g_cnfg.conv1_mp_size = 5
#g_cnfg.conv1_mp_strides = 2

#g_cnfg.conv2_n_filters = 32
#g_cnfg.conv2_kernel_size = 3
#g_cnfg.conv2_mp_size = 3
#g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [200, 400]
#g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

#g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
#g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
#g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 50
#g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_02_spectrogram_v2')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [5]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_img_h: 196
X_img_w: 105
Y_vector_len: 31
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [200, 400]
n_hidden1: 50

name: run_02_spectrogram_v2
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 367
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramMultiLSTM_graph_01_run_02_spectrogram_v2
Training starts @ 12/12/2017 18:04:08
Epoch 1 Step 1 Best model saved @ 12/12/2017 18:04:17 [Ave valid] 3.434
Epoch 1 Step 1 ends @ 12/12/2017 18:04:17 [Train] 3.432, 4.7% [Valid] 3.0% [Ave valid] 3.434
Epoch 1 Step 50 Best model saved @ 12/12/2017 18:04:32 [Ave valid] 3.425
Epoch 1 Step 100 Best model saved @ 12/12/2017 18:04:47 [Ave valid] 3.415
Epoch 1 Step 150 Best model saved @ 12/12/2017 18:05:03 [Ave valid] 3.404
Epoch 1 Step 200 Best model saved @ 12/12/2017 18:05:18 [Ave valid] 3.385
Epoch

Epoch 12 Step 4,250 Best model saved @ 12/12/2017 18:26:42 [Ave valid] 0.442
Epoch 12 Step 4,300 Best model saved @ 12/12/2017 18:26:56 [Ave valid] 0.436
Epoch 12 Step 4,400 Best model saved @ 12/12/2017 18:27:25 [Ave valid] 0.435
Epoch 12 Step 4,404 ends @ 12/12/2017 18:27:30 [Train] 0.362, 89.7% [Valid] 86.8% [Ave valid] 0.438
Epoch 13 Step 4,450 Best model saved @ 12/12/2017 18:27:46 [Ave valid] 0.434
Epoch 13 Step 4,500 Best model saved @ 12/12/2017 18:28:00 [Ave valid] 0.429
Epoch 13 Step 4,600 Best model saved @ 12/12/2017 18:28:29 [Ave valid] 0.429
Epoch 13 Step 4,700 Best model saved @ 12/12/2017 18:28:58 [Ave valid] 0.428
Epoch 13 Step 4,750 Best model saved @ 12/12/2017 18:29:12 [Ave valid] 0.424
Epoch 13 Step 4,771 Best model saved @ 12/12/2017 18:29:21 [Ave valid] 0.421
Epoch 13 Step 4,771 ends @ 12/12/2017 18:29:21 [Train] 0.245, 92.3% [Valid] 87.7% [Ave valid] 0.421
Epoch 14 Step 4,900 Best model saved @ 12/12/2017 18:30:05 [Ave valid] 0.414
Epoch 14 Step 4,950 Best model

Epoch 27 Step 9,800 Best model saved @ 12/12/2017 18:55:06 [Ave valid] 0.255
Epoch 27 Step 9,850 Best model saved @ 12/12/2017 18:55:20 [Ave valid] 0.255
Epoch 27 Step 9,900 Best model saved @ 12/12/2017 18:55:35 [Ave valid] 0.254
Epoch 27 Step 9,909 Best model saved @ 12/12/2017 18:55:40 [Ave valid] 0.253
Epoch 27 Step 9,909 ends @ 12/12/2017 18:55:40 [Train] 0.045, 100.0% [Valid] 92.9% [Ave valid] 0.253
Epoch 28 Step 9,950 Best model saved @ 12/12/2017 18:55:56 [Ave valid] 0.250
Epoch 28 Step 10,000 Best model saved @ 12/12/2017 18:56:10 [Ave valid] 0.249
Epoch 28 Step 10,050 Best model saved @ 12/12/2017 18:56:25 [Ave valid] 0.249
Epoch 28 Step 10,250 Best model saved @ 12/12/2017 18:57:21 [Ave valid] 0.247
Epoch 28 Step 10,276 ends @ 12/12/2017 18:57:30 [Train] 0.122, 96.6% [Valid] 92.7% [Ave valid] 0.247
Epoch 29 Step 10,350 Best model saved @ 12/12/2017 18:57:57 [Ave valid] 0.247
Epoch 29 Step 10,400 Best model saved @ 12/12/2017 18:58:11 [Ave valid] 0.246
Epoch 29 Step 10,450 Be

Epoch 53 Step 19,200 Best model saved @ 12/12/2017 19:42:35 [Ave valid] 0.196
Epoch 53 Step 19,451 ends @ 12/12/2017 19:43:49 [Train] 0.053, 98.3% [Valid] 94.7% [Ave valid] 0.201
Epoch 54 Step 19,818 ends @ 12/12/2017 19:45:37 [Train] 0.113, 97.4% [Valid] 94.7% [Ave valid] 0.198
Epoch 55 Step 19,900 Best model saved @ 12/12/2017 19:46:06 [Ave valid] 0.195
Epoch 55 Step 19,950 Best model saved @ 12/12/2017 19:46:20 [Ave valid] 0.195
Epoch 55 Step 20,185 ends @ 12/12/2017 19:47:27 [Train] 0.042, 98.3% [Valid] 94.6% [Ave valid] 0.197
Epoch 56 Step 20,552 ends @ 12/12/2017 19:49:19 [Train] 0.024, 99.1% [Valid] 94.6% [Ave valid] 0.196
Epoch 57 Step 20,600 Best model saved @ 12/12/2017 19:49:38 [Ave valid] 0.195
Epoch 57 Step 20,750 Best model saved @ 12/12/2017 19:50:19 [Ave valid] 0.194
Epoch 57 Step 20,800 Best model saved @ 12/12/2017 19:50:33 [Ave valid] 0.193
Epoch 57 Step 20,919 ends @ 12/12/2017 19:51:09 [Train] 0.043, 98.3% [Valid] 94.7% [Ave valid] 0.194
Epoch 58 Step 21,286 ends @

KeyboardInterrupt: 